<a href="https://colab.research.google.com/github/BenzterBit/DL-QSTP-Codes/blob/master/YOLOv3%20Trial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
#Getting configuration file for YOLO (Like downloading the pretrained network)
%%shell
mkdir cfg
cd cfg
wget https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg
pip install --upgrade wandb
# @hidden
wandb login 0ec07493250352b2c3a0596b81e4e91d1a2b9c48

--2019-07-15 13:45:26--  https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8342 (8.1K) [text/plain]
Saving to: ‘yolov3.cfg’

yolov3.cfg          100%[===================>]   8.15K  --.-KB/s    in 0s      

2019-07-15 13:45:26 (130 MB/s) - ‘yolov3.cfg’ saved [8342/8342]

Requirement already up-to-date: wandb in /usr/local/lib/python3.6/dist-packages (0.8.5)
Appending key for api.wandb.ai to your netrc file: /root/.netrc
Successfully logged in to Weights & Biases!


In [17]:
from __future__ import division
import torch 
import torch.nn as nn
import torch.nn.functional as F 
from torch.autograd import Variable
import numpy as np
import os
import wandb
wandb.init(project="yolo")
%cd cfg

/content/cfg/cfg


In [0]:
class EmptyLayer(nn.Module):
    def __init__(self):
        super(EmptyLayer, self).__init__()
        
class DetectionLayer(nn.Module):
    def __init__(self, anchors):
        super(DetectionLayer, self).__init__()
        self.anchors = anchors
        

In [0]:
def parse_cfg(directory="/content/cfg"):
  direct=os.listdir(directory)
  file = open(direct[1], 'r')
  lines = file.read().split('\n')                        # store the lines in a list
  lines = [x for x in lines if len(x) > 0]               # get read of the empty lines 
  lines = [x for x in lines if x[0] != '#']              # get rid of comments
  lines = [x.rstrip().lstrip() for x in lines]           # get rid of the extreme left and right spaces in the line
  block = {}
  blocks = []
  for line in lines:
      if line[0] == "[":               # This marks the start of a new block eg [convolutional]
          if len(block) != 0:          # If block is not empty, implies it is storing values of previous block
              blocks.append(block)     # add it the blocks list
              block = {}               # re-init the block
          block["type"] = line[1:-1].rstrip()     # [convolutional] ,so the block["type"] = made convolutional as '1' ignores '[' and :-1 ignores ']'
      else:                                         # append value to block
          key,value = line.split("=")  # splitting = , LHS = key , RHS = value ,only if it isn't the start of the block.
          block[key.rstrip()] = value.lstrip() # key is like convolutional , padding etc. removes the spaces between '=' and key and between '=' and value 
  blocks.append(block)                         # appending the last block
  return blocks

#so the flow works like first it checks whether the first letter is '[' if so goes inside the 'if' statement
#then it checks if the length of block is zero , if its not it means it contains the values from the previous block and should be appended to the block.
#else if the length of block is zero , then line would represent the following line 
#[convolutional]
#else if the first letter isn't '[' then it goes in the else part, splits line in key vlaue pair and stores it inside block variable
#as we reach the beginning of next block [convolutional]
#len(block)!=0 and the previous block is appended and then re initialized.
#at the end we do appending for the last block variable as it exits the 'for' loop
#and then we return the list 'blocks'


In [0]:
def create_modules(blocks):# To create the CNN
    net_info = blocks[0]     #Captures the information about the input and pre-processing , basically the [net] is initialized to it.   
    module_list = nn.ModuleList() #like an iterable module( list of modules) we need it dude to the large size (58) of number of layers
    prev_filters = 3 #coresponding to rgb channels, as a pre initialization for the first layer.
    output_filters = []
    for index, x in enumerate(blocks[1:]): # aside from the first block it captures the remaining blocks(58 in number) into module
      module = nn.Sequential() 
      
      
      if x["type"]=="convolutional" :
        try:
          batchnorm=int(x["batch_normalize"] )# if there is a batch norm present in the module then assign to variable batchnorm
          bias=False  # bias is +B part in calculation of NN , this gets canceled out in batch normalization so we do not require bias here
        except:
          batchnorm=0 #BN not present
          bias=True
        filters=int(x["filters"])
        kernel_size=int(x["size"])
        stride=int(x["stride"])
        padding=int(x["pad"]) # padding here is a boolean , it just says if padding is present or not , it is our job to calculate it
        if padding: # if present
          pad=(kernel_size-1)//2    #// is for division with output as integer  (why same padding???)
        else: 
          pad=0
        #torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')
        conv=nn.Conv2d(prev_filters, filters, kernel_size, stride=stride, padding=pad, bias=bias)
        module.add_module("conv_{0}".format(index), conv) #add module {index}

        if batchnorm:
          #torch.nn.BatchNorm2d(num_features, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          bn=nn.BatchNorm2d(filters)
          module.add_module("batch_norm_{0}".format(index), bn) #add module {index}
        if x["activation"]=="leaky":
          leaky_relu=nn.LeakyReLU(0.1,inplace=True)
          module.add_module("leaky_relu_{0}".format(index), leaky_relu) #add module {index}
          
          
      elif x["type"] == "upsample":       # if beginning of block/type is [upsample]
        stride = int(x["stride"])
        upsample = nn.Upsample(scale_factor = 2, mode = "bilinear") #We use Bilinear2dUpsampling
        module.add_module("upsample_{}".format(index), upsample)
            

      elif (x["type"] == "route"):            #If it is a route layer or beginning of block/type is [route]
          x["layers"] = x["layers"].split(',') # as layers can contain multiple values seperated by ','
          #Start  of a route
          start = int(x["layers"][0])          #first value of 'layers' 
          #end, if there exists one.
          try:#trying to access x["layers"][1] is access allowed then assign it to variable 'end'
              end = int(x["layers"][1])
          except:#if exception occurs and access is denied
              end = 0
          #Positive anotation
          if start > 0: 
              start = start - index            #subtracting index if start is greater than 0. not applicable in this config
          if end > 0:
              end = end - index               #subtraction index if end is greater than 0. applicable is this config.
          route = EmptyLayer()
          module.add_module("route_{0}".format(index), route) #add an emptylayer irrespective of the previous operations
          if end < 0:              # if only one value in layers, then the output 'x' will be the feature map of i+start 'th layer
              filters = output_filters[index + start] + output_filters[index + end]  #else if 2 values in layers, add 2 filters i+start and i+end 
          else:
              filters= output_filters[index + start]
#followubg is the code from the darknet(nn.Module)
#thats why we make route 'empty' layer so that we can directly access it in darknet class' forward method.
              '''
elif module_type == "route":     #if module type is root
  layers = modules[i]["layers"]  #access the "layers" of the i'th module
  layers = [int(a) for a in layers] #converting layers to 'int'
                
if (layers[0]) > 0:               #equivalnet to start > 0
  layers[0] = layers[0] - i
                    
if len(layers) == 1:
  x = outputs[i + (layers[0])]   # if only one value in layers, then the output 'x' will be the feature map of i+layers[0] 'th layer
                    

else:
  if (layers[1]) > 0:            #equivalent explaination to end>0
    layers[1] = layers[1] - i
                        
  map1 = outputs[i + layers[0]]
  map2 = outputs[i + layers[1]]                    
                    
  x = torch.cat((map1, map2), 1) #concat along the depth dimension 
outputs[i] = x                  #assign 'x' to this layer output.
'''  

              
      elif x["type"] == "shortcut":
        shortcut = EmptyLayer()
        module.add_module("shortcut_{}".format(index), shortcut)
        '''
elif  module_type == "shortcut":
  from_ = int(modules[i]["from"])
  x = outputs[i-1] + outputs[i+from_]
  outputs[i] = x 
  
'''
#Yolo is the detection layer
      elif x["type"] == "yolo":
        mask = x["mask"].split(",") 
        mask = [int(x) for x in mask]

        anchors = x["anchors"].split(",") #the spaces in between each pair won't matter inside the list.
        anchors = [int(a) for a in anchors]
        anchors = [(anchors[i], anchors[i+1]) for i in range(0, len(anchors),2)]
        anchors = [anchors[i] for i in mask]

        detection = DetectionLayer(anchors)
        module.add_module("Detection_{}".format(index), detection)
        
      module_list.append(module)
      prev_filters = filters
      output_filters.append(filters)
    return (net_info, module_list)


In [0]:
from __future__ import division

import torch 
import torch.nn as nn
import torch.nn.functional as F 
from torch.autograd import Variable
import numpy as np
import cv2 

In [0]:
def predict_transform(prediction, inp_dim, anchors, num_classes, CUDA = True):
  batch_size = prediction.size(0)
  stride =  inp_dim // prediction.size(2)
  grid_size = inp_dim // stride
  bbox_attrs = 5 + num_classes
  num_anchors = len(anchors)

  prediction = prediction.view(batch_size, bbox_attrs*num_anchors, grid_size*grid_size)
  prediction = prediction.transpose(1,2).contiguous()
  prediction = prediction.view(batch_size, grid_size*grid_size*num_anchors, bbox_attrs)
  
  anchors = [(a[0]/stride, a[1]/stride) for a in anchors]
  #Sigmoid the  centre_X, centre_Y. and object confidencce
  prediction[:,:,0] = torch.sigmoid(prediction[:,:,0])
  prediction[:,:,1] = torch.sigmoid(prediction[:,:,1])
  prediction[:,:,4] = torch.sigmoid(prediction[:,:,4])
  
  #Add the center offsets
  grid = np.arange(grid_size)
  a,b = np.meshgrid(grid, grid)

  x_offset = torch.FloatTensor(a).view(-1,1)
  y_offset = torch.FloatTensor(b).view(-1,1)

  if CUDA:
      x_offset = x_offset.cuda()
      y_offset = y_offset.cuda()

  x_y_offset = torch.cat((x_offset, y_offset), 1).repeat(1,num_anchors).view(-1,2).unsqueeze(0)

  prediction[:,:,:2] += x_y_offset
  #log space transform height and the width
  anchors = torch.FloatTensor(anchors)

  if CUDA:
      anchors = anchors.cuda()

  anchors = anchors.repeat(grid_size*grid_size, 1).unsqueeze(0)
  prediction[:,:,2:4] = torch.exp(prediction[:,:,2:4])*anchors
  
  prediction[:,:,5: 5 + num_classes] = torch.sigmoid((prediction[:,:, 5 : 5 + num_classes]))
  
  prediction[:,:,:4] *= stride
  return prediction


  


In [0]:
class Darknet(nn.Module):
    def __init__(self):
      super(Darknet, self).__init__()
      self.blocks = parse_cfg()
      self.net_info, self.module_list = create_modules(self.blocks)
    def forward(self,x,CUDA):
      modules = self.blocks[1:]  #as the first module is network information and has no role in the forward pass
      outputs = {}               #as for the route and shortcut layers we would need the output feature maps of previous layers, hence we cache 
                                  #all of them in the dictionary 'outputs'
      write = 0     #This is explained a bit later
      for i, module in enumerate(modules): #iterating over the modules       
        module_type = (module["type"])
        if module_type == "convolutional" or module_type == "upsample": #if the 'type' is [convolutional] or [upsample]
          x = self.module_list[i](x)    #here 'x' is the input to darknet which will come in forthcoming snippets
        elif module_type == "route":     #if module type is root
          layers = modules[i]["layers"]  #access the "layers" of the i'th module
          layers = [int(a) for a in layers] #converting layers to 'int'

          if (layers[0]) > 0:               #equivalnet to start > 0 that comes in create_module function
            layers[0] = layers[0] - i

          if len(layers) == 1:
            x = outputs[i + (layers[0])]   # if only one value in layers, then the output 'x' will be the feature map of i+layers[0] 'th layer


          else:
            if (layers[1]) > 0:            #equivalent explaination to end>0 that comes in create_module function
              layers[1] = layers[1] - i

            map1 = outputs[i + layers[0]]  #creating the 2 feature map variables for concatenation. 
            map2 = outputs[i + layers[1]]                    

            x = torch.cat((map1, map2), 1) #concat along the depth dimension 

        elif  module_type == "shortcut":
            from_ = int(module["from"])
            x = outputs[i-1] + outputs[i+from_] #as shortcut layer does addition of the previous and the layer before the skip connection node.
            
        elif module_type == 'yolo':        # if type = [yolo]
          anchors = self.module_list[i][0].anchors  #assigns the value of 'anchors' from conv2d ?? in i'th module || DOUBT:NO ANCHORS in module_list
          #Get the input dimensions
          inp_dim = int (self.net_info["height"])  #assigns the value of 'height' from [net] 

          #Get the number of classes
          num_classes = int (module["classes"])   #assigns the value of classes from module || DOUBT:NO CLASSES IN MODULE

          #Transform 
          x = x.data   #data which goes inside the yolo layer
          x = predict_transform(x, inp_dim, anchors, num_classes, CUDA)
          if not write:              #if no collector has been intialised. 
              detections = x
              write = 1

          else:       
              detections = torch.cat((detections, x), 1)

        outputs[i] = x
      return detections
      

      

In [24]:
!wget https://github.com/ayooshkathuria/pytorch-yolo-v3/raw/master/dog-cycle-car.png

--2019-07-15 13:45:36--  https://github.com/ayooshkathuria/pytorch-yolo-v3/raw/master/dog-cycle-car.png
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ayooshkathuria/pytorch-yolo-v3/master/dog-cycle-car.png [following]
--2019-07-15 13:45:37--  https://raw.githubusercontent.com/ayooshkathuria/pytorch-yolo-v3/master/dog-cycle-car.png
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 347445 (339K) [image/png]
Saving to: ‘dog-cycle-car.png’

dog-cycle-car.png   100%[===================>] 339.30K  --.-KB/s    in 0.05s   

2019-07-15 13:45:37 (6.64 MB/s) - ‘dog-cycle-car.png’ saved [347445/347445]



In [0]:
def get_test_input(): 
  img = cv2.imread("/content/dog-cycle-car.png")
  img = cv2.resize(img, (608,608))          #Resize to the input dimension
  img_ =  img[:,:,::-1].transpose((2,0,1))  # BGR -> RGB | H X W C -> C X H X W 
  img_ = img_[np.newaxis,:,:,:]/255.0       #Add a channel at 0 (for batch) | Normalise
  img_ = torch.from_numpy(img_).float()     #Convert to float
  img_ = Variable(img_)                     # Convert to Variable
  return img_


In [26]:
!ls

dog-cycle-car.png  yolov3.cfg


In [27]:
model = Darknet()
inp = get_test_input()
pred = model(inp, torch.cuda.is_available())
print (pred)

FileNotFoundError: ignored